# Homework 3 Factor Analysis
**Due Date: 11/14/2017**

Name: Gosuddin Siddiqi<br>
Student Number: 1627383

## Problem 2: Convex low rank matrix factorization
Consider the problem,
$$\min_X \frac{1}{2}\|R - X\|_F^2 + \lambda\|X\|_*$$

In [1]:
# Generate the sythetic data
srand(123);
m = 500;
n = 200;
R = randn(m,n);
λ = 30.0;

In [2]:
# Define functions
# objective function
function cvx_mf_obj(X)
    # TODO: complete the objective function
    U,Σ,v = svd(X)
    return (0.5*(vecnorm(R-X)^2) + λ*sum(Σ))
end

# gradient function
function cvx_mf_smooth_grad(X)
    # TODO: return gradient
    return(X-R)
end

# prox of ‖⋅‖_*
function prox_nuclear_norm(Z, α)
    # TODO: return the prox solution
    U,Σ,V = svd(Z)
    prox_v = zeros(Σ)
    for i = 1:length(Σ)
        if Σ[i] > α
            prox_v[i] = Σ[i] - α
        end
    end
    return (U*diagm(prox_v))*transpose(V)
end

prox_nuclear_norm (generic function with 1 method)

In [3]:
# proximal gradient method
η   = 1.0;
tol = 1e-6;
itm = 1000;
err = Inf;

X   = copy(R);
X⁻  = copy(X);
for iter = 1:itm
    # TODO: Implement proximal gradient descent method
    
    X⁻ = copy(X)
    ∇ = cvx_mf_smooth_grad(X)
    X = prox_nuclear_norm( (X - η .* ∇), λ.*η)
    
    
    obj = cvx_mf_obj(X)
    err = vecnorm(X-X⁻) / η
    
    @printf("iter %4d, obj %1.2e, err %1.2e\n", iter, obj, err);
    err < tol && break;
end

iter    1, obj 5.00e+04, err 3.09e+02
iter    2, obj 5.00e+04, err 2.65e-13


In [4]:
# analysis the result
k = rank(X)
U,S,V = svd(R);
X_k = U[:,1:k]*diagm(S[1:k])*(V[:,1:k]).';

In [5]:
k

29

## Problem 3: Nonconvex low rank matrix factorization
Consider problem,
$$\min_{B,F} \frac{1}{2}\|R - BF\|_F^2.$$

In [47]:
# Define functions
# objective
function noncvx_mf_obj(B,F)
    # TODO: return objective funciton
    return 0.5*(vecnorm(R - (B*F)) ^2)
end

# gradient
function noncvx_mf_grad_B(B,F)
    # TODO: return gradient function w.r.t. B
    return ((B*F - R)*transpose(F))
end

function noncvx_mf_grad_F(B,F)
    # TODO: return gradient function w.r.t. F
    return (transpose(B)*(B*F - R))
end

noncvx_mf_grad_F (generic function with 1 method)

In [49]:
# PALM algorithm
tol = 1e-6;
itm = 30000;
err = Inf;

B   = randn(m,k);
B⁻  = copy(B);
F   = randn(k,n);
F⁻  = copy(F);

g_B = noncvx_mf_grad_B(B,F)
g_F = noncvx_mf_grad_F(B,F)

for iter = 1:itm
    # update B
    # TODO: what is step size for B should be?
    #       implement 1 gradient step
    η_B = 0.0
    β_B = vecnorm(F⁻*transpose(F⁻))
    η_B = 1.0/β_B
    B = B⁻ - (η_B*g_B)
    
    # update F
    # TODO: what is step size for F should be?
    #       implement 1 gradient step
    η_F = 0.0
    β_F = vecnorm(transpose(B)*B)
    η_F = 1.0/β_F
    F = F⁻ - (η_F * g_F)
    
    
    # update convergent history
    err = vecnorm(B - B⁻) + vecnorm(F - F⁻);
    g_B = noncvx_mf_grad_B(B,F)
    g_F = noncvx_mf_grad_F(B,F)
    obj = noncvx_mf_obj(B, F);

    copy!(B⁻, B);
    copy!(F⁻, F);
    iter % 1000 == 0 && @printf("iter %5d, obj %1.2e, err %1.2e\n", iter, obj, err);
    err < tol && break;
end

iter  1000, obj 3.47e+04, err 2.66e-02
iter  2000, obj 3.47e+04, err 9.55e-03
iter  3000, obj 3.47e+04, err 4.81e-03
iter  4000, obj 3.47e+04, err 2.66e-03
iter  5000, obj 3.47e+04, err 1.53e-03
iter  6000, obj 3.47e+04, err 9.01e-04
iter  7000, obj 3.47e+04, err 5.36e-04
iter  8000, obj 3.47e+04, err 3.22e-04
iter  9000, obj 3.47e+04, err 1.94e-04
iter 10000, obj 3.47e+04, err 1.18e-04
iter 11000, obj 3.47e+04, err 7.23e-05
iter 12000, obj 3.47e+04, err 4.44e-05
iter 13000, obj 3.47e+04, err 2.74e-05
iter 14000, obj 3.47e+04, err 1.70e-05
iter 15000, obj 3.47e+04, err 1.06e-05
iter 16000, obj 3.47e+04, err 6.71e-06
iter 17000, obj 3.47e+04, err 4.28e-06
iter 18000, obj 3.47e+04, err 2.76e-06
iter 19000, obj 3.47e+04, err 1.80e-06
iter 20000, obj 3.47e+04, err 1.19e-06


In [50]:
# compare result
@show vecnorm(X_k - B*F)/vecnorm(X_k)

vecnorm(X_k - B * F) / vecnorm(X_k) = 3.6788657894983514e-5


3.6788657894983514e-5

The value looks almost the same but only after 30000 iterations

## Problem 4: Robust Risk Decomposition
Consider problem,
$$\min_{B,F} \rho_\kappa(\bar{R} - BF), \quad\text{s.t. }B^TB = I.$$

In [6]:
# load data
N   = 499;
T   = 251;
fid = open("Data/returns.bin", "r");
R   = read(fid, Float64, (N,T));
close(fid);
R̄ = broadcast(-, R, mean(R,2));
@show size(R̄)
κ = 1.0;
itm = 8000;
tol = 1e-6;

size(R̄) = (499,251)


In [7]:
# TODO: implement PALM to solve the objective
#       you could follow the steps we are using so far
#       * define functions

function huber(r)
    h = 0.0
    for I in eachindex(r)
        h = 0.0
        if abs(r[I]) > κ
            h += (abs(r[I])*κ) - (0.5* (κ^2));
        else
            h += 0.5 * (abs(r[I])^2);
        end
    end
    
    return h

end


function huber_∇(x)
    for I in eachindex(x)
        if x[I]< -κ
            x[I] = -κ
        elseif x[I] > κ
            x[I] = κ
        end
    end
    return x

end

function robust(B,F)
    return (huber(R̄ - (B*F)))
end


function grad_B(B,F)
    r = R̄ - (B * F)
    h_d = huber_∇(r)
    derv_B = - (h_d * transpose(F))
    return derv_B
end
    
function grad_F(B,F)
    r = R̄ - (B * F)
    h_d = huber_∇(r)
    derv_F = - (transpose(B)*h_d)
    return derv_F
end
    
function prox_svd(Z)
    U, Σ, V = svd(Z)
    return (U * transpose(V))

end



prox_svd (generic function with 1 method)

In [8]:
#       * implement solver
tol = 1e-6;
itm = 30000;
err = Inf;

B   = randn(N,k);
B⁻  = copy(B);
F   = randn(k,T);
F⁻  = copy(F);

rd_∇_B = grad_B(B,F)
rd_∇_F = grad_F(B,F)


for iter = 1:itm
    # update B
    # TODO: what is step size for B should be?
    #       implement 1 gradient step
    η_B = 0.0
    β_B = (vecnorm(F⁻))^2
    η_B = 1.0/β_B
    B = prox_svd(B⁻ - (η_B*rd_∇_B))
    
    # update F
    # TODO: what is step size for F should be?
    #       implement 1 gradient step
    η_F = 1.0
    F = F⁻ - (η_F * rd_∇_F)
    
    
    # update convergent history
    err = vecnorm(B - B⁻) + vecnorm(F - F⁻);
    rd_∇_B = grad_B(B,F)
    rd_∇_F = grad_F(B,F)
    obj = robust(B, F);

    copy!(B⁻, B);
    copy!(F⁻, F);
    iter % 1000 == 0 && @printf("iter %5d, obj %1.2e, err %1.2e\n", iter, obj, err);
    err < tol && break;
end

iter  1000, obj 4.81e-02, err 2.48e+00
iter  2000, obj 4.59e-02, err 3.53e-02
iter  3000, obj 2.49e-02, err 2.54e-02
iter  4000, obj 1.97e-02, err 2.06e-02
iter  5000, obj 1.82e-02, err 1.57e-02
iter  6000, obj 1.94e-02, err 1.27e-02
iter  7000, obj 2.09e-02, err 1.27e-02
iter  8000, obj 2.18e-02, err 9.77e-03
iter  9000, obj 2.24e-02, err 8.93e-03
iter 10000, obj 2.25e-02, err 9.77e-03
iter 11000, obj 2.25e-02, err 1.20e-02
iter 12000, obj 2.24e-02, err 1.40e-02
iter 13000, obj 2.21e-02, err 1.38e-02
iter 14000, obj 2.14e-02, err 1.34e-02
iter 15000, obj 2.10e-02, err 1.92e-02
iter 16000, obj 2.17e-02, err 1.17e-02
iter 17000, obj 2.17e-02, err 8.31e-03
iter 18000, obj 2.11e-02, err 7.95e-03
iter 19000, obj 2.06e-02, err 6.11e-03
iter 20000, obj 2.03e-02, err 4.76e-03
iter 21000, obj 2.04e-02, err 3.97e-03
iter 22000, obj 2.06e-02, err 3.80e-03
iter 23000, obj 2.09e-02, err 3.87e-03
iter 24000, obj 2.11e-02, err 3.91e-03
iter 25000, obj 2.15e-02, err 3.17e-03
iter 26000, obj 2.19e-02,

In [9]:
# extract the portfolio
b1  = B[:,1];
ind = indmax(abs(b1));
@show ind

ind = 30


30